In [77]:
import webbrowser
import os
import parsel
from urllib.request import urlopen
from lxml.html import fromstring, tostring
from IPython.display import IFrame
from IPython.core.display import display, HTML
from IPython.display import clear_output
from ipywidgets import widgets

class interactive(object):

    def __init__(self):
        clear_output()
        self.htmlsample = ''

    def get_page(self, url):
        html = urlopen(url).read()
        dom = fromstring(html)
        dom.make_links_absolute(url)
        return dom

    def analyze(self, url):
        dom = self.get_page(url)
        self.htmlsample = tostring(dom).decode('utf-8')

    def cleanup(self, x):
        x = [i.strip() for i in x]
        x = [i for i in x if i != '']
        return x

    def show(self, css_selector):
        doc = parsel.Selector(text=self.htmlsample)
        selector_list = doc.css(css_selector)
        x = selector_list.extract()
        x = self.cleanup(x)
        y = ['<style> .parselinteractive {background-color: yellow; font-weight:bold;} </style>' + 
             '<span class="parselinteractive">' +
             i + '</span>' for i in x]
        
        newsample = self.htmlsample

        for i, j in zip(x, y):
            newsample = newsample.replace(i, j)

        newpath = os.path.realpath('./modified.html')

        with open(newpath, 'w') as f:
            f.write(newsample)

        display(IFrame('./modified.html', width=900, height=1000))


website = widgets.Text(description="Website to scrape", width=200)
display(website)

css_selector = widgets.Text(description="CSS Selector", width=200)
display(css_selector)

button = widgets.Button(description="Check the selector")
display(button)

def on_button_clicked(b):
    obj = interactive()
    obj.analyze(str(website.value))
    obj.show(css_selector.value)

button.on_click(on_button_clicked)